# Project 1 Starter

**This is draft - version 0 - changes are possible and will be anounced.**

Project 1 is to allow students to practice Data Science concepts learned so far.

The project will include following tasks:
- Load dataset. Don't use "index" column for training.
- Clean up the data:
    - Encode replace missing values
    - Replace features values that appear incorrect
- Encode categorical variables
- Split dataset to Train/Validation/Test
- Add engineered features
- Train and tune ML model
- Provide final metrics using Test dataset

### Types of models to train

Your final submission should include single model. 
The model set you should try to come up with best model:
1. Sklearn Logistic Regression - try all combinations of regularization
2. H2O-3 GLM - try different combinations of regularization



### Feature engineering

You should train/fit categorical features scalers and encoders on Train only. Use `transform` or equivalent function on Validation/Test datasets.

It is important to understand all the steps before model training, so that you can reliably replicate and test them to produce scoring function.


You should generate various new features. Examples of such features can be seen in the Module-3 lecture on GLMs.
Your final model should have at least **10** new engineered features. On-hot-encoding, label encoding, and target encoding is not included in the **10** features.
You can try, but target encoding is not expected to produce improvement for Linear models.

Ideas for Feature engineering for various types of variables:
1. https://docs.h2o.ai/driverless-ai/1-10-lts/docs/userguide/transformations.html
2. GLM lecture and hands-on (Module-3)


**Note**: 
- You don't have to perform feature engineering using H2O-3 even if you decided to use H2O-3 GLM for model training.
- It is OK to perfor feature engineering using any technique, as long as you can replicate it correctly in the Scoring function.


### Threshold calculation

You will need to calculate optimal threshold for class assignment using F1 metric:
- If using sklearn, use F1 `macro`: `f1_score(y_true, y_pred, average='macro')` 
- If using H2O-3, use F1

You will need to find optimal probability threshold for class assignment, the threshold that maximizes above F1.



### Scoring function

The Project-1 will be graded based on the completeness and performance of your final model against the hold-out dataset.
The hold-out dataset will not be known to the students. As part of your deliverables, you will need to submit a scoring function. The scoring function will perform the following:
- Accept dataset in the same format as provided with the project, minus "MIS_Status" column
- Load trained model and any encoders/scalers that are needed to transform data
- Transform dataset into format that can be scored with the trained model
- Score the dataset and return the results, for each record
    - Record ID
    - Record label as determined by final model (0 or 1)
    - If your model returns probabilities, you need to assign the label based on maximum F1 threshold
    
Scoring function header:
```
def project_1_scoring(data):
    """
    Function to score input dataset.
    
    Input: dataset in Pandas DataFrame format
    Output: Python list of labels in the same order as input records
    
    Flow:
        - Load artifacts
        - Transform dataset
        - Score dataset
        - Return labels
    
    """
    l = data.shape[0]
    return l*[0]
```

Look for full example of scoring function at the bottom of the notebook. **Don't copy as is - this is just an example**



### Deliverables in a single zip file in the following structure:
- `notebook` (folder)
    - Jupyter notebook with complete code to manipulate data, train and tune final model. `ipynb` format
    - Jupyter notebook in `html` format
- `artifacts` (folder)
    - Model and any potential encoders in the "pkl" format or native H2O-3 format (for H2O-3 model)
    - Scoring function that will load the final model and encoders. Separate from above notebook or `.py` file



Your notebook should include explanations about your code and be designed to be easily followed and results replicated. Once you are done with the final version, you will need to test it by running all cells from top to bottom after restarting Kernel. It can be done by running `Kernel -> Restart & Run All`


**Important**: To speed up progress, first produce working code using a small subset of the dataset.

## Dataset description

The dataset is from the U.S. Small Business Administration (SBA) The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market (SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment. There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.  
More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied

**Don't use original dataset, use only dataset provided with project requirements in eLearning**

## Preparation

Use dataset provided in the eLearning

In [6]:
import pandas as pd
pd.set_option('display.max_columns', 1500)

import warnings
warnings.filterwarnings('ignore')

#Extend cell width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [45]:
"""
Created on Mon Mar 18 18:25:50 2019

@author: Uri Smashnov

Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:100].to_dict())
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

In [46]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Zulu17.34+19-CA (build 17.0.3+7-LTS, mixed mode, sharing)
  Starting server from C:\Users\win10\anaconda3\envs\battlezone\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\win10\AppData\Local\Temp\tmprprfmkhm
  JVM stdout: C:\Users\win10\AppData\Local\Temp\tmprprfmkhm\h2o_win10_started_from_python.out
  JVM stderr: C:\Users\win10\AppData\Local\Temp\tmprprfmkhm\h2o_win10_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,23 days
H2O_cluster_name:,H2O_from_python_win10_wu9c0t
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.973 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


### Load data

In [19]:
full_data = pd.read_csv('SBA_loans_project_1.zip')

In [21]:
print("Data shape:", full_data.shape)

Data shape: (809247, 21)


**Review dataset**

In [22]:
desc_df = describe_more(full_data)
desc_df

,A_Variable,Levels,Datatype,Min Length,Max Length,Level_Values
0,index,809247,int64,1,6,"{0: 1, 539491: 1, 539493: 1, 539494: 1, 539495..."
1,City,31320,object,1,30,"{'LOS ANGELES': 10372, 'HOUSTON': 9260, 'NEW Y..."
2,State,51,object,2,3,"{'CA': 117341, 'TX': 63425, 'NY': 51877, 'FL':..."
3,Zip,32731,int64,1,5,"{10001: 841, 90015: 830, 93401: 729, 90010: 65..."
4,Bank,5716,object,3,30,"{'BANK OF AMERICA NATL ASSOC': 78111, 'WELLS F..."
5,BankState,55,object,2,3,"{'CA': 106293, 'NC': 71557, 'IL': 59258, 'OH':..."
6,NAICS,1307,int64,1,6,"{0: 181845, 722110: 25217, 722211: 17476, 8111..."
7,Term,407,int64,1,3,"{84: 207228, 60: 80965, 240: 77385, 120: 69852..."
8,NoEmp,581,int64,1,4,"{1: 138836, 2: 124470, 3: 81466, 4: 66306, 5: ..."
9,NewExist,3,float64,3,3,"{1.0: 580478, 2.0: 227709, 0.0: 932}"


## Dataset preparation and clean-up

Modify and clean-up the dataset as following:
- Replace encode Na/Null values
- Convert the strings styled as '$XXXX.XX' to float values. Columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
- Convert MIS_Status to 0/1. Make value "CHGOFF" as 1

Any additional clean-up as you find fit.

In [23]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809247 entries, 0 to 809246
Data columns (total 21 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   index              809247 non-null  int64  
 1   City               809222 non-null  object 
 2   State              809235 non-null  object 
 3   Zip                809247 non-null  int64  
 4   Bank               807842 non-null  object 
 5   BankState          807836 non-null  object 
 6   NAICS              809247 non-null  int64  
 7   Term               809247 non-null  int64  
 8   NoEmp              809247 non-null  int64  
 9   NewExist           809119 non-null  float64
 10  CreateJob          809247 non-null  int64  
 11  RetainedJob        809247 non-null  int64  
 12  FranchiseCode      809247 non-null  int64  
 13  UrbanRural         809247 non-null  int64  
 14  RevLineCr          805153 non-null  object 
 15  LowDoc             806928 non-null  object 
 16  Di

In [24]:
full_data.isna().sum()

index                   0
City                   25
State                  12
Zip                     0
Bank                 1405
BankState            1411
NAICS                   0
Term                    0
NoEmp                   0
NewExist              128
CreateJob               0
RetainedJob             0
FranchiseCode           0
UrbanRural              0
RevLineCr            4094
LowDoc               2319
DisbursementGross       0
BalanceGross            0
GrAppv                  0
SBA_Appv                0
MIS_Status           1822
dtype: int64

In [113]:
data = full_data.sample(n=100000)

In [114]:
# Imputing missing values

data['NewExist'].fillna(method='ffill', inplace=True)

In [115]:
data[['City', 'State', 'Bank', 'BankState', 'RevLineCr', 'LowDoc']] = \
    data[['City', 'State', 'Bank', 'BankState', 'RevLineCr', 'LowDoc']].fillna(value="Missing")

In [117]:
str_cols = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
data[str_cols] = data[str_cols].apply(lambda x: x.str.replace('$', '').str.replace(',', '').astype(float))

C:\Users\win10\AppData\Local\Temp\ipykernel_3452\3284344168.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[str_cols] = data[str_cols].apply(lambda x: x.str.replace('$', '').str.replace(',', '').astype(float))


In [116]:
# dropping unwanted columns
data.drop(axis=1, columns=['City', 'Zip', 'Bank', 'index'], inplace=True)

# dropping the rows with null values in the output column
data.dropna(axis=0, inplace=True)

In [118]:
# imputing MIS_Status column
data['MIS_Status'] = data['MIS_Status'].apply(lambda x: 1 if x=='CHGOFF' else 0)

## Categorical and numerical variables encoding

Encode categorical variables using either one of the techniques below. Don't use LabelEncoder.
- One-hot-encoder for variables with less than 10 valid values. Name your new columns "Original_name"_valid_value
- Target encoder from the following library: https://contrib.scikit-learn.org/category_encoders/index.html . Name your new column "Original_name"_trg
- WOE encoder from the following library: https://contrib.scikit-learn.org/category_encoders/index.html . Name your new column "Original_name"_woe


WOE encoder can be used with numerical variables too. 


Example of use for target encoder:
```
import category_encoders as ce

encoder = ce.TargetEncoder(cols=[...])

encoder.fit(X, y)
X_cleaned = encoder.transform(X_dirty)
```

In [119]:
# onehot encoding columns
onehot = ['NewExist', 'UrbanRural', 'RevLineCr', 'LowDoc']
data = pd.get_dummies(data, columns=onehot, prefix=onehot, drop_first=True)

In [120]:
# splitting
train, valid, test = data.iloc[:70000,:], data.iloc[:84000,:], data.iloc[84000:,:]

In [121]:
# Applying WOE encoding
from category_encoders.woe import WOEEncoder

woe_cols = ['BankState', 'State']

# creating an instance
woe = WOEEncoder(cols=woe_cols, random_state=123)

# fitting to the training data
woe.fit(X=train[woe_cols], y=train["MIS_Status"])

# transforming train, test and valid data
train_ = woe.transform(train[woe_cols]).add_suffix('_woe')
valid_ = woe.transform(valid[woe_cols]).add_suffix('_woe')
test_ = woe.transform(test[woe_cols]).add_suffix('_woe')

train = train.join(train_)
train.drop(columns=woe_cols, inplace=True)
valid = valid.join(valid_)
valid.drop(columns=woe_cols, inplace=True)
test = test.join(test_)
test.drop(columns=woe_cols, inplace=True)

In [122]:
def add_features(data, inter_ls):
    interactions = [(feat1, feat2) for feat1 in inter_ls 
                    for feat2 in inter_ls if feat1 != feat2]
    
    for feat1, feat2 in interactions:
        col_name = feat1 + '_x_' + feat2
        data[col_name] = data[feat1] * data[feat2]
                
    return data

In [123]:
inter_ls = ['Term','NoEmp','CreateJob','RetainedJob','FranchiseCode']

train = add_features(train, inter_ls)
test = add_features(test, inter_ls)
valid = add_features(valid, inter_ls)

# Model Training

See Project summary for types of models

In [124]:
# importing 
from sklearn.linear_model import LogisticRegression

# Define the logistic regression model
lr = LogisticRegression(random_state=42, max_iter=500)

# Train the model on the training set
X = train.drop('MIS_Status', axis=1)
y = train['MIS_Status']
lr.fit(X, y)

# Scoring the model
from sklearn.metrics import f1_score

valid_pred = lr.predict(valid.drop('MIS_Status', axis=1))
test_pred = lr.predict(test.drop('MIS_Status', axis=1))

print("Val_score: ", f1_score(valid['MIS_Status'], valid_pred, average='macro'))
print("Test_score: ", f1_score(test['MIS_Status'], test_pred, average='macro'))

Val_score:  0.45609195710968287
Test_score:  0.45612077345136215


C:\Users\win10\anaconda3\envs\battlezone\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [125]:
# importing
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

# Convert the data to an H2OFrame
train_h2o = h2o.H2OFrame(train)
valid_h2o = h2o.H2OFrame(valid)
test_h2o = h2o.H2OFrame(test)

# Defining predictors and response columns
predictors = train_h2o.columns
response = 'MIS_Status'
predictors.remove(response)

# Define the GLM model and train it
glm_model = H2OGeneralizedLinearEstimator(family='binomial')
glm_model.train(x=predictors, 
                y=response, 
                training_frame=train_h2o, 
                validation_frame=valid_h2o)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
glm Model Build progress: |

C:\Users\win10\anaconda3\envs\battlezone\Lib\site-packages\h2o\estimators\estimator_base.py:193: RuntimeWarning: We have detected that your response column has only 2 unique values (0/1). If you wish to train a binary model instead of a regression model, convert your target column to categorical before training.
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_model_python_1680461535637_1


GLM Model: summary
    family    link    regularization                                 number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  ------  ---------------------------------------------  ----------------------------  -----------------------------  ----------------------  ------------------------------------------------------
    binomial  logit   Elastic Net (alpha = 0.5, lambda = 2.425E-4 )  53                            38                             5                       Key_Frame__upload_ad89de9ac0c0d28d38560e659bb0479f.hex

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.1054203168376448
RMSE: 0.32468495012495546
LogLoss: 0.35449702515585757
AUC: 0.8429058113829855
AUCPR: 0.5820867011181713
Gini: 0.685811622765971
Null degrees of freedom: 69999
Residual degrees of freedom: 69961
Null deviance: 65418.43788498982
Residual deviance: 49629.58352182007
AIC: 49707.58352182007

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.289574087155084
       0      1      Error    Rate
-----  -----  -----  -------  -----------------
0      50141  7448   0.1293   (7448.0/57589.0)
1      4202   8209   0.3386   (4202.0/12411.0)
Total  54343  15657  0.1664   (11650.0/70000.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.289574     0.584937  199
max f2                       0.20183      0.668471  250
max f0point5                 0.410674     0.611835  142
max accuracy                 0.410674     0.863557  142
max precision                0.996039     0.866667  0
max recall                   0.000543579  1         399
max specificity              0.996039     0.999965  0
max absolute_mcc             0.341757     0.490047  174
max min_per_class_accuracy   0.216347     0.774635  241
max mean_per_class_accuracy  0.23175      0.778075  232
max tns                      0.996039     57587     0
max fns                      0.996039     12398     0
max fps                      0.000543579  57589     399
max tps                      0.000543579  12411     399
max tnr                      0.996039     0.999965  0
max fnr                      0.996039     0.998953  0
max fpr                      0.000543579  1         399
max tpr                      0.000543579  1         399

Gains/Lift Table: Avg response rate: 17.73 %, avg score: 17.73 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.01                        0.722874           4.43961    4.43961            0.787143         0.788369    0.787143                    0.788369            0.0443961       0.0443961                  343.961   343.961            0.0418088
2        0.02                        0.663326           4.35904    4.39932            0.772857         0.691421    0.78                        0.739895            0.0435904       0.0879865                  335.904   339.932            0.0826382
3        0.03                        0.620707           4.35904    4.38589            0.772857         0.641511    0.777619                    0.7071              0.0435904       0.131577                   335.904   338.589            0.123468
4        0.04                        0.58659            4.18983    4.33688            0.742857         0.

In [126]:
# Scoring the model
glm_model.F1(train=True, valid=True)

{'train': [[0.289574087155084, 0.5849365825851504]],
 'valid': [[0.29130094499946446, 0.5830041891083184]]}

## Model Tuning

Choose one model from the above list. You should provide reasoning on why you have picked the model over others. Perform tuning for the selected model:
- Hyper-parameter tuning. Your hyper-parameter search space should have at least 50 combinations.
- To avoid overfitting and provide you with reasonable estimate of model performance on hold-out dataset, you will need to split your dataset as following:
    - Train, will be used to train model
    - Validation, will be used to validate model each round of training
    - Testing, will be used to provide final performance metrics, used only once on the final model
- Feature engineering. See project description

**Selelct final model that produces best performance on the Test dataset.**
- For the best model, calculate probability threshold to maximize F1. 

In [127]:
# Explanation for model selection
# For model tuning, the H2O-3 GLM model is used since it has a higher F1 score when compared to Logistic regression 

In [128]:
# importing
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch

# Define the hyperparameters to search over
hyper_params = {'alpha': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
                'lambda': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05]}

# Define the GLM model and search grid
glm_model = H2OGeneralizedLinearEstimator(family='binomial', 
                                          nfolds=5)
glm_grid = H2OGridSearch(model=glm_model, 
                         hyper_params=hyper_params,
                         grid_id="glm_grid")

# Train the grid search
glm_grid.train(x=predictors, 
               y=response, 
               training_frame=train_h2o, 
               validation_frame=valid_h2o)

glm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


C:\Users\win10\anaconda3\envs\battlezone\Lib\site-packages\h2o\job.py:83: UserWarning: Reached maximum number of iterations 7!
  warnings.warn(w)
C:\Users\win10\anaconda3\envs\battlezone\Lib\site-packages\h2o\job.py:83: UserWarning: Reached maximum number of iterations 6!
  warnings.warn(w)
C:\Users\win10\anaconda3\envs\battlezone\Lib\site-packages\h2o\job.py:83: UserWarning: Reached maximum number of iterations 5!
  warnings.warn(w)
C:\Users\win10\anaconda3\envs\battlezone\Lib\site-packages\h2o\grid\grid_search.py:431: UserWarning: Adding alpha array to hyperparameter runs slower with gridsearch. This is due to the fact that the algo has to run initialization for every alpha value. Setting the alpha array as a model parameter will skip the initialization and run faster overall.
  warnings.warn(w_message)


,alpha,lambda,model_ids,logloss
,0.9,0.0001,glm_grid_model_9,0.3549524
,0.3,0.0001,glm_grid_model_3,0.3549573
,0.6,0.0001,glm_grid_model_6,0.3550029
,0.8,0.0001,glm_grid_model_8,0.3550195
,0.5,0.0001,glm_grid_model_5,0.3550196
,0.4,0.0001,glm_grid_model_4,0.3550255
,0.2,0.0001,glm_grid_model_2,0.3551005
,0.7,0.0001,glm_grid_model_7,0.3551406
,0.4,0.0005,glm_grid_model_13,0.3551909
,0.3,0.0005,glm_grid_model_12,0.3552480


In [129]:
res = glm_grid.get_grid(sort_by='f1', decreasing=True)
best_model = res.models[0]

In [130]:
from sklearn.metrics import roc_curve

# getting the predicted probability of class 1
prob_1 = best_model.predict(test_h2o)["p1"]

# converting h2oFrame to list
prob_1 = prob_1.as_data_frame().values.tolist()
true_y = test["MIS_Status"].values.tolist()

# getting results from ROC curve
fpr, tpr, thresholds = roc_curve(true_y, prob_1)

# Calculate index for the best probability threshold
best_f1 = 0
best_f1_id = 0
for id, threshold in enumerate(thresholds):
    f1 = f1_score(prob_1 >= threshold, true_y)
    if f1 > best_f1:
        best_f1 = f1
        best_f1_id = id

# getting the best
best_threshold = thresholds[best_f1_id]

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [131]:
best_threshold, best_f1

(0.2358352136841801, 0.6033789219629928)

## Save all artifacts

Save all artifacts needed for scoring function:
- Trained model
- Encoders
- Any other arficats you will need for scoring

**You should stop your notebook here. Scoring function should be in a separate file/notebook.**

## Stop Here. Create new file/notebook

## ==============================================

In [132]:
# importing
import pickle

# Saving the encoder using pickle
with open('woe.pkl', 'wb') as f:
    pickle.dump(woe, f)
    
# Save the trained model
best_model.save_mojo("model.zip")

# other artifacts
dict_artifacts = {'thres' : best_threshold,
                  'cat_cols' : ['City', 'State', 'Bank', 'BankState', 'RevLineCr', 'LowDoc'],
                  'str_cols' : ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv'],
                  'drop_cols' : ['City', 'Zip', 'Bank', 'index'],
                  'oh_cols' : ['NewExist', 'UrbanRural', 'RevLineCr', 'LowDoc'],
                  'woe_cols' : ['BankState', 'State'],
                  'interaction_list' : ['Term','NoEmp','CreateJob','RetainedJob','FranchiseCode']}

with open('dict_artifacts.pkl', 'wb') as f:
    pickle.dump(dict_artifacts, f)

## Model Scoring

Write function that will load artifacts from above, transform and score on a new dataset.
Your function should return Python list of labels. For example: [0,1,0,1,1,0,0]


In [2]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM Zulu17.34+19-CA (build 17.0.3+7-LTS, mixed mode, sharing)
  Starting server from C:\Users\win10\anaconda3\envs\battlezone\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\win10\AppData\Local\Temp\tmpgc476dbb
  JVM stdout: C:\Users\win10\AppData\Local\Temp\tmpgc476dbb\h2o_win10_started_from_python.out
  JVM stderr: C:\Users\win10\AppData\Local\Temp\tmpgc476dbb\h2o_win10_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.2
H2O_cluster_version_age:,24 days
H2O_cluster_name:,H2O_from_python_win10_ytgjqk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,5.973 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [3]:
def project_1_scoring(data):
    """
    Function to score input dataset.
    
    Input: dataset in Pandas DataFrame format
    Output: Python list of labels in the same order as input records
    
    Flow:
        - Load artifacts
        - Transform dataset
        - Score dataset
        - Return pandas DF with following columns:
            - index
            - label
            - probability_0
            - probability_1
    """
    pass

In [4]:
def add_features(data, inter_ls):
    interactions = [(feat1, feat2) for feat1 in inter_ls 
                    for feat2 in inter_ls if feat1 != feat2]
    
    for feat1, feat2 in interactions:
        col_name = feat1 + '_x_' + feat2
        data[col_name] = data[feat1] * data[feat2]
                
    return data

In [11]:
def project_1_scoring(data):
    '''
    Data: pandas.DataFrame
    '''
    # importing
    import pickle
    
    # importing artifacts
    # data encoding
    with open('woe.pkl', 'rb') as f:
        woe = pickle.load(f)
    # importing the h2o model
    model = h2o.import_mojo("model.zip")
    # importing artifacts dictionary
    with open('dict_artifacts.pkl', 'rb') as f:
        dict_artifacts = pickle.load(f)
    
    # data preprocessing
    data["NewExist"].fillna(method='ffill', inplace=True)
    
    data[dict_artifacts[cat_cols]] = \
        data[dict_artifacts[cat_cols]].fillna(value="Missing")
    
#     cat_cols = ['City', 'State', 'Bank', 'BankState', 'RevLineCr', 'LowDoc']
#     for col in dict_artifacts[cat_cols]:
#         data[col].fillna(value="Missing", inplace=True)
        
    str_cols = dict_artifacts[str_cols]
    data[str_cols] = data[str_cols].apply(lambda x: x.str.replace('$', '').str.replace(',', '').astype(float))
    
#     str_cols = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
#     for col in dict_artifacts[str_cols]:
#         data[col] = data[col].str.replace('$', '')
#         data[col] = data[col].str.replace(',', '').astype(float)
        
    data['MIS_Status'] = data['MIS_Status'].apply(lambda x: 1 if x=='CHGOFF' else 0)
    
#     oh_cols = ['NewExist', 'UrbanRural', 'RevLineCr', 'LowDoc']
#     for col in oh_cols:
#         int_data = pd.get_dummies(data[col], prefix=col, drop_first=True)
#         data = pd.concat([data, int_data], axis=1)
#         data.drop(columns=[col], axis=1, inplace=True)

    onehot = dict_artifacts[str_cols]
    data = pd.get_dummies(data, columns=onehot, 
                          prefix=onehot, drop_first=True)
    
    data.drop(axis=1, columns=['City', 'Zip', 'Bank', 'BalanceGross', 'index', 'MIS_Status'], inplace=True)
    
    # data encoding
    woe_cols = ['BankState', 'State']
    woe_data = woe.transform(data[woe_cols])
    woe_data.set_axis(labels=['BankState_woe', 'State_woe'], axis=1, inplace=True)
    data = pd.concat([data, woe_data], axis=1)
    data.drop(columns=woe_cols, axis=1, inplace=True)
    
    # feature engineering
    interaction_list = ['Term','NoEmp','CreateJob','RetainedJob','FranchiseCode']
    data = create_features(data, interaction_list)
    
    # Convert the data to an H2OFrame
    data = h2o.H2OFrame(data)
    
    # predicting from the model
    res = model.predict(data)
    prob_1 = res.as_data_frame()
    y_pred = [1 if x > 0.352094 else 0 for x in prob_1['p1']]
    
    d = {
        'index' : [i for i in range(1, len(data)+1)],
        'label' : y_pred,
        'probability_0' : prob_1['p0'],
        'probability_1' : prob_1['p1']
    }

    return pd.DataFrame(d)

In [12]:
import pandas as pd

data = pd.read_csv('SBA_loans_project_1.zip')
project_1_scoring(data)

generic Model Build progress: | (failed)


OSError: Job with key $03017f00000132d4ffffffff$_867bddfcdaf9228222c185925f944f07 failed with an exception: java.lang.AssertionError
stacktrace: 
java.lang.AssertionError
	at hex.generic.Generic.importFile(Generic.java:152)
	at hex.generic.Generic.access$000(Generic.java:23)
	at hex.generic.Generic$MojoDelegatingModelDriver.compute2(Generic.java:90)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1677)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:976)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1479)
	at jsr166y.ForkJoinWorkerThread.run(ForkJoinWorkerThread.java:104)


### Example of Scoring function

Don't copy the code as is. It is provided as an example only. 
- Function `train_model` - you need to focus on model and artifacts saving:
    ```
    pickle.dump(obj=artifacts_dict, file=artifacts_dict_file)
    ```
- Function `project_1_scoring` - you should have similar function with name `project_1_scoring`. The function will:
    - Get Pandas dataframe as parameter
    - Will load model and all needed encoders
    - Will perform needed manipulations on the input Pandas DF - in the exact same format as input file for the project, minus MIS_Status feature
    - Return Pandas DataFrame
        - record index
        - predicted class for threshold maximizing F1
        - probability for class 0 (PIF)
        - probability for class 1 (CHGOFF)


In [7]:
"""
Don't copy of use the cell code in any way!!!
The code is provided as an example of generating artifacts for scoring function
Your scoring function code should not have model training part!!!!
"""
import pandas as pd
import numpy as np
def train_model(data):
    """
    Train sample model and save artifacts
    """
    from sklearn.preprocessing import OneHotEncoder
    from copy import deepcopy
    from sklearn.linear_model import LogisticRegression
    import pickle
    from sklearn.impute import SimpleImputer
    
    target_col = "Survived"
    cols_to_drop = ['Name', 'Ticket', 'Cabin','SibSp', 'Parch', 'Sex','Embarked','PassengerId','Survived']
    y = data[target_col]
    X = data.drop(columns=[target_col])
    
    # Impute Embarked
    X['Embarked'].replace(np.NaN, 'S',inplace = True)
    
    # Create new feature
    X['FamilySize'] = X['SibSp'] + X['Parch']
    
    # Mean impute Age
    imp_age_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    imp_age_mean.fit(X[['Age']])
    X['Age'] = imp_age_mean.transform(X[['Age']])


    ohe_orig_columns = ["Embarked","Sex"]
    cat_encoders = {}
    for col in ohe_orig_columns:
        enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
        enc.fit(X[[col]])
        result = enc.transform(X[[col]])
        ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
        result_train = pd.DataFrame(result, columns=ohe_columns)
        X= pd.concat([X, result_train], axis=1)
        cat_encoders[col] = [deepcopy(enc),"ohe"]
        
    clf = LogisticRegression(max_iter=1000, random_state=0)
    
    columns_to_train = [x for x in X.columns if x not in cols_to_drop]
    clf.fit(X[columns_to_train], y)
    
    # Todo: Add code to calculate optimal threshold. Replace 0.5 !!!!!
    threshold = 0.5
    # End Todo
    
    artifacts_dict = {
        "model": clf,
        "cat_encoders": cat_encoders,
        "imp_age_mean": imp_age_mean,
        "ohe_columns": ohe_orig_columns,
        "columns_to_train":columns_to_train,
        "threshold": threshold
    }
    artifacts_dict_file = open("./artifacts/artifacts_dict_file.pkl", "wb")
    pickle.dump(obj=artifacts_dict, file=artifacts_dict_file)
    
    artifacts_dict_file.close()    
    return clf

In [8]:
df_train = pd.read_csv('titanic.csv')
train_model(df_train)

LogisticRegression(max_iter=1000, random_state=0)

### Example scoring function

This is example only. Don't copy code as is!!!   
You must place scoring function in a separate Python file or Jupyter notebook.   

**Don't place function in the same notebook as rest of the code**

In [9]:
def project_1_scoring(data):
    """
    Function to score input dataset.
    
    Input: dataset in Pandas DataFrame format
    Output: Python list of labels in the same order as input records
    
    Flow:
        - Load artifacts
        - Transform dataset
        - Score dataset
        - Return labels
    
    """
    from sklearn.preprocessing import OneHotEncoder
    from copy import deepcopy
    from sklearn.linear_model import LogisticRegression
    import pickle
    
    X = data.copy()
    
    '''Load Artifacts'''
    artifacts_dict_file = open("./artifacts/artifacts_dict_file.pkl", "rb")
    artifacts_dict = pickle.load(file=artifacts_dict_file)
    artifacts_dict_file.close()
    
    clf = artifacts_dict["model"]
    cat_encoders = artifacts_dict["cat_encoders"]
    imp_age_mean = artifacts_dict["imp_age_mean"]
    ohe_columns = artifacts_dict["ohe_columns"]
    columns_to_score = artifacts_dict["columns_to_train"]
    threshold = artifacts_dict["threshold"]
    
    # Impute Embarked
    X['Embarked'].replace(np.NaN, 'S',inplace = True)
    
    # Create new feature
    X['FamilySize'] = X['SibSp'] + X['Parch']
    
    # Mean impute Age
    X['Age'] = imp_age_mean.transform(X[['Age']])
    
    '''Encode categorical columns'''
    for col in ohe_columns:
        enc = cat_encoders[col][0]
        result = enc.transform(X[[col]])
        ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]]
        result_train = pd.DataFrame(result, columns=ohe_columns)
        X = pd.concat([X, result_train], axis=1)
        
    y_pred_proba = clf.predict_proba(X[columns_to_score])
    y_pred = (y_pred_proba[:,0] < threshold).astype(np.int16)
    d = {"index":data["PassengerId"],
         "label":y_pred,
         "probability_0":y_pred_proba[:,0],
         "probability_1":y_pred_proba[:,1]}
    
    return pd.DataFrame(d)

In [10]:
project_1_scoring(df_train).head()

,index,label,probability_0,probability_1
0,1,0,0.901298,0.098702
1,2,1,0.071879,0.928121
2,3,1,0.367665,0.632335
3,4,1,0.098564,0.901436
4,5,0,0.923460,0.076540
